In [19]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

## 법정동 정리

In [20]:
bjd = pd.read_csv("bupjeongdong.csv")

In [21]:
bjd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


https://www.code.go.kr/stdcode/regCodeL.do

In [22]:
l = []
for c in bjd.법정동코드:
    l.append(len(str(c)))
pd.DataFrame(l).value_counts()

10    46292
dtype: int64

In [23]:
str(bjd.법정동코드[0])[:5]

'11000'

In [24]:
list(map(lambda x:str.split()))

TypeError: map() must have at least two arguments.

In [25]:
bjd["시군구"] = list(map(lambda x: str(x)[:5] , bjd.법정동코드))
bjd["법정동"] = list(map(lambda x: str(x)[5:] , bjd.법정동코드))

In [26]:
bjd

,법정동코드,법정동명,폐지여부,시군구,법정동
0,1100000000,서울특별시,존재,11000,00000
1,1111000000,서울특별시 종로구,존재,11110,00000
2,1111010100,서울특별시 종로구 청운동,존재,11110,10100
3,1111010200,서울특별시 종로구 신교동,존재,11110,10200
4,1111010300,서울특별시 종로구 궁정동,존재,11110,10300
...,...,...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재,50130,32022
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재,50130,32023
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재,50130,32024
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재,50130,32025


In [27]:
address = list(map(lambda x: x.split(" ") , bjd.법정동명))

In [28]:
pd.DataFrame(list(map(lambda x: x[0] , address))).value_counts()

경상북도       9224
경기도        6522
경상남도       6093
충청남도       5267
전라남도       4821
전라북도       3350
충청북도       3147
강원도        2839
서울특별시      1112
부산직할시       454
대구광역시       368
제주도         357
인천광역시       344
대구직할시       334
부산광역시       297
광주직할시       272
울산광역시       248
광주광역시       246
인천직할시       223
대전직할시       220
제주특별자치도     216
대전광역시       184
세종특별자치시     154
dtype: int64

In [29]:
bjd["시군구1"] = list(map(lambda x: x[0] , address))
bjd["시군구2"] = list(map(lambda x: x[1] if len(x)>1 else '' , address))
bjd["법정동1"] = list(map(lambda x: x[2] if len(x)>2 else '' , address))
bjd["법정동2"] = list(map(lambda x: x[3] if len(x)>3 else '' , address))

In [30]:
bjd

,법정동코드,법정동명,폐지여부,시군구,법정동,시군구1,시군구2,법정동1,법정동2
0,1100000000,서울특별시,존재,11000,00000,서울특별시,,,
1,1111000000,서울특별시 종로구,존재,11110,00000,서울특별시,종로구,,
2,1111010100,서울특별시 종로구 청운동,존재,11110,10100,서울특별시,종로구,청운동,
3,1111010200,서울특별시 종로구 신교동,존재,11110,10200,서울특별시,종로구,신교동,
4,1111010300,서울특별시 종로구 궁정동,존재,11110,10300,서울특별시,종로구,궁정동,
...,...,...,...,...,...,...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재,50130,32022,제주특별자치도,서귀포시,표선면,하천리
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재,50130,32023,제주특별자치도,서귀포시,표선면,성읍리
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재,50130,32024,제주특별자치도,서귀포시,표선면,가시리
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재,50130,32025,제주특별자치도,서귀포시,표선면,세화리


## 법정동 test set 과 merge

In [31]:
exdata = pd.read_csv("Daesamanlap/External/external_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Daesamanlap/External/external_data.csv'

In [ ]:
test.head()

In [ ]:
ad.head()

In [ ]:
test.isnull().sum()

In [ ]:
exdata[exdata.주택유형명 == "아파트"]

In [ ]:
test.도로명주소[0]+"\n"

In [ ]:
f = open("도로명주소.txt", 'w')
for a in list(test.도로명주소):
    data = a+"\n"
    f.write(data)
f.close()

https://cafe.naver.com/excelmaster/152667

In [34]:
pop1 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106주민등록인구및세대현황_월간.xlsx")
pop2 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106연령별인구현황_월간.xlsx")

In [35]:
tr = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/train_processed.csv")
tr

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역,버스정류장,단지내주차면수,등록차량수,단지명,도로명주소
0,C2483,900,아파트,경상북도,국민임대,39.7200,134,38.0000,A,"15,667,000.0000","103,680.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
1,C2483,900,아파트,경상북도,국민임대,39.7200,15,38.0000,A,"15,667,000.0000","103,680.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
2,C2483,900,아파트,경상북도,국민임대,51.9300,385,38.0000,A,"27,304,000.0000","184,330.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
3,C2483,900,아파트,경상북도,국민임대,51.9300,15,38.0000,A,"27,304,000.0000","184,330.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
4,C2483,900,아파트,경상북도,국민임대,51.9300,41,38.0000,A,"27,304,000.0000","184,330.0000",0.0000,3.0000,"1,425.0000","1,015.0000",포항장량휴먼시아1단지,경상북도 포항시 북구 장량중앙로 17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153
2948,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153
2949,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153
2950,C2571,481,상가,경상남도,임대상가,31.8400,1,5.0000,D,"5,580,000.0000","71,900.0000",0.0000,1.0000,65.0000,62.0000,통영도남주공아파트,경상남도 통영시 발개로 153


In [36]:
pop1.head()

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1100000000,서울특별시,"9,565,990","4,405,638",2.1700,"4,649,442","4,916,548",0.9500
1,1111000000,서울특별시 종로구,"146,029","73,525",1.9900,"70,985","75,044",0.9500
2,1111051500,서울특별시 종로구 청운효자동,"12,024","5,242",2.2900,"5,580","6,444",0.8700
3,1111053000,서울특별시 종로구 사직동,"9,371","4,674",2.0000,"4,225","5,146",0.8200
4,1111054000,서울특별시 종로구 삼청동,"2,498","1,248",2.0000,"1,191","1,307",0.9100


In [37]:
pop2.head(2)

,행정기관코드,행정기관,총인구수,총연령구간인구수,총0~9세,총10~19세,총20~29세,총30~39세,총40~49세,총50~59세,총60~69세,총70~79세,총80~89세,총90~99세,총100세이상,남 인구수,남연령구간인구수,남0~9세,남10~19세,남20~29세,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상
0,1100000000,서울특별시,"9,565,990","9,565,990","602,526","766,572","1,443,208","1,452,328","1,501,071","1,506,232","1,269,804","696,049","284,923","40,808","2,469","4,649,442","4,649,442","309,269","392,686","696,930",...,"743,762","741,675","599,745","317,147","111,573","10,322",619,"4,916,548","4,916,548","293,257","373,886","746,278","726,614","757,309","764,557","670,059","378,902","173,350","30,486","1,850"
1,1111000000,서울특별시 종로구,"146,029","146,029","7,275","10,684","23,147","19,595","21,433","24,939","20,192","11,791","5,985",936,52,"70,985","70,985","3,651","5,311","11,255",...,"10,450","12,579","10,055","5,224","2,413",288,16,"75,044","75,044","3,624","5,373","11,892","9,852","10,983","12,360","10,137","6,567","3,572",648,36


In [38]:
print(bjd.shape)
print(pop1.shape)
print(pop2.shape)

(46292, 9)
(3846, 8)
(3847, 41)


In [39]:
bjd1 = bjd.merge(pop1, left_on = ["법정동코드"], right_on =["행정기관코드"],how = "inner")
bjd1

,법정동코드,법정동명,폐지여부,시군구,법정동,시군구1,시군구2,법정동1,법정동2,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1100000000,서울특별시,존재,11000,00000,서울특별시,,,,1100000000,서울특별시,"9,565,990","4,405,638",2.1700,"4,649,442","4,916,548",0.9500
1,1111000000,서울특별시 종로구,존재,11110,00000,서울특별시,종로구,,,1111000000,서울특별시 종로구,"146,029","73,525",1.9900,"70,985","75,044",0.9500
2,1114000000,서울특별시 중구,존재,11140,00000,서울특별시,중구,,,1114000000,서울특별시 중구,"123,016","62,897",1.9600,"60,019","62,997",0.9500
3,1117000000,서울특별시 용산구,존재,11170,00000,서울특별시,용산구,,,1117000000,서울특별시 용산구,"226,378","111,650",2.0300,"108,948","117,430",0.9300
4,1120000000,서울특별시 성동구,존재,11200,00000,서울특별시,성동구,,,1120000000,서울특별시 성동구,"289,162","134,542",2.1500,"141,042","148,120",0.9500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,5013025000,제주특별자치도 서귀포시 대정읍,존재,50130,25000,제주특별자치도,서귀포시,대정읍,,5013025000,제주특별자치도 서귀포시 대정읍,"21,959","10,523",2.0900,"10,687","11,272",0.9500
1679,5013025300,제주특별자치도 서귀포시 남원읍,존재,50130,25300,제주특별자치도,서귀포시,남원읍,,5013025300,제주특별자치도 서귀포시 남원읍,"18,717","8,677",2.1600,"9,668","9,049",1.0700
1680,5013025900,제주특별자치도 서귀포시 성산읍,존재,50130,25900,제주특별자치도,서귀포시,성산읍,,5013025900,제주특별자치도 서귀포시 성산읍,"15,478","8,165",1.9000,"8,105","7,373",1.1000
1681,5013031000,제주특별자치도 서귀포시 안덕면,존재,50130,31000,제주특별자치도,서귀포시,안덕면,,5013031000,제주특별자치도 서귀포시 안덕면,"12,205","6,288",1.9400,"6,306","5,899",1.0700


In [40]:
bjd2 = bjd.merge(pop1, left_on = ["법정동명"], right_on =["행정기관"],how = "inner")
bjd2

,법정동코드,법정동명,폐지여부,시군구,법정동,시군구1,시군구2,법정동1,법정동2,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1111011500,서울특별시 종로구 사직동,존재,11110,11500,서울특별시,종로구,사직동,,1111053000,서울특별시 종로구 사직동,"9,371","4,674",2.0000,"4,225","5,146",0.8200
1,1111014000,서울특별시 종로구 삼청동,존재,11110,14000,서울특별시,종로구,삼청동,,1111054000,서울특별시 종로구 삼청동,"2,498","1,248",2.0000,"1,191","1,307",0.9100
2,1111014600,서울특별시 종로구 가회동,존재,11110,14600,서울특별시,종로구,가회동,,1111060000,서울특별시 종로구 가회동,"4,210","2,030",2.0700,"1,944","2,266",0.8600
3,1111016500,서울특별시 종로구 이화동,존재,11110,16500,서울특별시,종로구,이화동,,1111064000,서울특별시 종로구 이화동,"7,347","4,289",1.7100,"3,472","3,875",0.9000
4,1111016900,서울특별시 종로구 혜화동,존재,11110,16900,서울특별시,종로구,혜화동,,1111065000,서울특별시 종로구 혜화동,"16,489","9,194",1.7900,"7,949","8,540",0.9300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,5013025000,제주특별자치도 서귀포시 대정읍,존재,50130,25000,제주특별자치도,서귀포시,대정읍,,5013025000,제주특별자치도 서귀포시 대정읍,"21,959","10,523",2.0900,"10,687","11,272",0.9500
2230,5013025300,제주특별자치도 서귀포시 남원읍,존재,50130,25300,제주특별자치도,서귀포시,남원읍,,5013025300,제주특별자치도 서귀포시 남원읍,"18,717","8,677",2.1600,"9,668","9,049",1.0700
2231,5013025900,제주특별자치도 서귀포시 성산읍,존재,50130,25900,제주특별자치도,서귀포시,성산읍,,5013025900,제주특별자치도 서귀포시 성산읍,"15,478","8,165",1.9000,"8,105","7,373",1.1000
2232,5013031000,제주특별자치도 서귀포시 안덕면,존재,50130,31000,제주특별자치도,서귀포시,안덕면,,5013031000,제주특별자치도 서귀포시 안덕면,"12,205","6,288",1.9400,"6,306","5,899",1.0700


In [41]:
len(set(bjd1.법정동코드) | set(bjd2.법정동코드))

2523

In [42]:
len(set(bjd2.법정동코드)- set(bjd1.법정동코드))

840

In [49]:
bjd3 = pd.concat([bjd1,bjd2],axis=0).drop_duplicates()

In [44]:
bjd.법정동명[46100:46200]

46100        제주특별자치도 제주시 외도일동
46101        제주특별자치도 제주시 외도이동
46102        제주특별자치도 제주시 이호일동
46103        제주특별자치도 제주시 이호이동
46104        제주특별자치도 제주시 도두일동
46105        제주특별자치도 제주시 도두이동
46106         제주특별자치도 제주시 도남동
46107        제주특별자치도 제주시 도련일동
46108        제주특별자치도 제주시 도련이동
46109         제주특별자치도 제주시 용강동
46110         제주특별자치도 제주시 회천동
46111         제주특별자치도 제주시 오등동
46112         제주특별자치도 제주시 월평동
46113         제주특별자치도 제주시 영평동
46114          제주특별자치도 제주시 연동
46115         제주특별자치도 제주시 도평동
46116         제주특별자치도 제주시 해안동
46117         제주특별자치도 제주시 내도동
46118         제주특별자치도 제주시 한림읍
46119     제주특별자치도 제주시 한림읍 귀덕리
46120     제주특별자치도 제주시 한림읍 수원리
46121     제주특별자치도 제주시 한림읍 대림리
46122     제주특별자치도 제주시 한림읍 한림리
46123     제주특별자치도 제주시 한림읍 금악리
46124     제주특별자치도 제주시 한림읍 동명리
46125     제주특별자치도 제주시 한림읍 명월리
46126     제주특별자치도 제주시 한림읍 옹포리
46127     제주특별자치도 제주시 한림읍 상대리
46128     제주특별자치도 제주시 한림읍 상명리
46129     제주특별자치도 제주시 한림읍 협재리
46130     제주특별자치도 제주시 한림읍 월령리
46131     제주특별자치도 제주시 한림읍 금능리
46132     제주특별자치도 제주시 한림읍 월림리
46133     

## MERGE

In [50]:
bld_tr = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/External3/building_tr.csv")
bld_tst = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/External3/building_tst.csv")

In [51]:
bld_tr

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수,가구수,도로명주소,지번주소
0,47113-100216867,장량휴먼시아,0.0000,632.9550,"8,409.5540","8,394.6490",0.0000,149,0,경상북도 포항시 북구 장량중앙로 17,경상북도 포항시 북구 양덕동 336-1
1,48220-100186682,미수휴먼시아,0.0000,661.9600,"1,213.8700",240.8300,0.0000,0,0,경상남도 통영시 미우지해안로 107,경상남도 통영시 미수동 695
2,30200-100200744,도안휴먼시아4단지,0.0000,707.1500,"4,636.4100","4,636.4100",0.0000,102,0,대전광역시 유성구 상대로 40,대전광역시 유성구 상대동 465
3,41360-100216608,휴먼시아,0.0000,385.9900,"6,017.7600","5,951.0800",0.0000,76,0,경기도 남양주시 늘을3로 8,경기도 남양주시 호평동 712
4,45140-100190554,장신휴먼시아1단지,0.0000,556.1800,"6,611.9000","6,611.9000",0.0000,84,0,전라북도 익산시 오산면 선화로4길 66,전라북도 익산시 오산면 장신리 1130
...,...,...,...,...,...,...,...,...,...,...,...
348,50130-100340536,서귀포서홍 행복주택,0.0000,0.0000,67.9800,0.0000,0.0000,0,0,제주특별자치도 서귀포시 장수로 115,제주특별자치도 서귀포시 서홍동 355-1
349,42110-100311797,춘천우두엘에이치천년나무3단지,0.0000,0.0000,59.4100,59.4100,0.0000,0,0,강원도 춘천시 우두1길 40,강원도 춘천시 우두동 1088
350,26290-100212522,용호행복주택,0.0000,0.0000,152.2900,0.0000,0.0000,0,0,부산광역시 남구 동명로 183,부산광역시 남구 용호동 376-9
351,43150-100289809,제천 공공실버주택,0.0000,0.0000,88.8600,0.0000,0.0000,0,0,충청북도 제천시 용두천로40길 28,충청북도 제천시 고암동 1273


In [52]:
train = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/train_processed.csv")

In [53]:
train.도로명주소.nunique()

421

In [92]:
juso = list(map(lambda x: ''.join(x.split()[:-1]) , bld_tr.지번주소))
juso

['경상북도포항시북구양덕동',
 '경상남도통영시미수동',
 '대전광역시유성구상대동',
 '경기도남양주시호평동',
 '전라북도익산시오산면장신리',
 '전라북도익산시오산면장신리',
 '경기도성남시분당구백현동',
 '대전광역시유성구지족동',
 '강원도횡성군횡성읍읍하리',
 '광주광역시남구백운동',
 '광주광역시남구백운동',
 '강원도강릉시교동',
 '강원도강릉시포남동',
 '강원도강릉시입암동',
 '경상남도거제시장평동',
 '충청남도당진시석문면통정리',
 '충청남도공주시옥룡동',
 '부산광역시기장군기장읍교리',
 '경상남도김해시구산동',
 '경상남도김해시삼문동',
 '대전광역시서구관저동',
 '대전광역시서구관저동',
 '대전광역시서구월평동',
 '대전광역시중구문화동',
 '대전광역시유성구반석동',
 '대전광역시대덕구법동',
 '대전광역시중구중촌동',
 '경상남도창원시마산회원구내서읍삼계리',
 '경상남도창원시마산회원구내서읍상곡리',
 '경상남도밀양시가곡동',
 '경상남도밀양시삼문동',
 '충청남도보령시명천동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구덕천동',
 '부산광역시북구만덕동',
 '부산광역시사상구모라동',
 '부산광역시해운대구반송동',
 '부산광역시북구금곡동',
 '충청남도부여군부여읍쌍북리',
 '경상남도사천시벌리동',
 '충청남도서산시석림동',
 '강원도속초시교동',
 '울산광역시북구화봉동',
 '강원도원주시문막읍건등리',
 '부산광역시기장군정관읍용수리',
 '제주특별자치도제주시아라일동',
 '제주특별자치도제주시노형동',
 '제주특별자치도제주시화북일동',
 '경상남도진주시가좌동',
 '경상남도진주시평거동',
 '경상남도창원시진해구자은동',
 '경상남도창녕군창녕읍말흘리',
 '충청남도천안시서북구백석동',
 '충청남도천안시서북구성정동',
 '충청남도천안시서북구쌍용동',
 '충청남도청양군청양읍읍내리',
 '강원도춘천시후평동',
 '강원도태백시철암동',
 '경상남도통영시도남동',


In [93]:
juso = list(map(lambda x: x.split()[:-1] , bld_tr.지번주소))
juso

[['경상북도', '포항시', '북구', '양덕동'],
 ['경상남도', '통영시', '미수동'],
 ['대전광역시', '유성구', '상대동'],
 ['경기도', '남양주시', '호평동'],
 ['전라북도', '익산시', '오산면', '장신리'],
 ['전라북도', '익산시', '오산면', '장신리'],
 ['경기도', '성남시', '분당구', '백현동'],
 ['대전광역시', '유성구', '지족동'],
 ['강원도', '횡성군', '횡성읍', '읍하리'],
 ['광주광역시', '남구', '백운동'],
 ['광주광역시', '남구', '백운동'],
 ['강원도', '강릉시', '교동'],
 ['강원도', '강릉시', '포남동'],
 ['강원도', '강릉시', '입암동'],
 ['경상남도', '거제시', '장평동'],
 ['충청남도', '당진시', '석문면', '통정리'],
 ['충청남도', '공주시', '옥룡동'],
 ['부산광역시', '기장군', '기장읍', '교리'],
 ['경상남도', '김해시', '구산동'],
 ['경상남도', '김해시', '삼문동'],
 ['대전광역시', '서구', '관저동'],
 ['대전광역시', '서구', '관저동'],
 ['대전광역시', '서구', '월평동'],
 ['대전광역시', '중구', '문화동'],
 ['대전광역시', '유성구', '반석동'],
 ['대전광역시', '대덕구', '법동'],
 ['대전광역시', '중구', '중촌동'],
 ['경상남도', '창원시', '마산회원구', '내서읍', '삼계리'],
 ['경상남도', '창원시', '마산회원구', '내서읍', '상곡리'],
 ['경상남도', '밀양시', '가곡동'],
 ['경상남도', '밀양시', '삼문동'],
 ['충청남도', '보령시', '명천동'],
 ['부산광역시', '북구', '금곡동'],
 ['부산광역시', '북구', '금곡동'],
 ['부산광역시', '북구', '금곡동'],
 ['부산광역시', '북구', '금곡동'],
 ['부산광역시', '북구', '덕천동']

In [94]:
for idx, j in enumerate (juso):
    if j[-1].strip()[-1] == "리":
        juso[idx] = j[:-1]

In [95]:
juso = list(map(lambda x: "".join(x) , juso))
juso

['경상북도포항시북구양덕동',
 '경상남도통영시미수동',
 '대전광역시유성구상대동',
 '경기도남양주시호평동',
 '전라북도익산시오산면',
 '전라북도익산시오산면',
 '경기도성남시분당구백현동',
 '대전광역시유성구지족동',
 '강원도횡성군횡성읍',
 '광주광역시남구백운동',
 '광주광역시남구백운동',
 '강원도강릉시교동',
 '강원도강릉시포남동',
 '강원도강릉시입암동',
 '경상남도거제시장평동',
 '충청남도당진시석문면',
 '충청남도공주시옥룡동',
 '부산광역시기장군기장읍',
 '경상남도김해시구산동',
 '경상남도김해시삼문동',
 '대전광역시서구관저동',
 '대전광역시서구관저동',
 '대전광역시서구월평동',
 '대전광역시중구문화동',
 '대전광역시유성구반석동',
 '대전광역시대덕구법동',
 '대전광역시중구중촌동',
 '경상남도창원시마산회원구내서읍',
 '경상남도창원시마산회원구내서읍',
 '경상남도밀양시가곡동',
 '경상남도밀양시삼문동',
 '충청남도보령시명천동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구덕천동',
 '부산광역시북구만덕동',
 '부산광역시사상구모라동',
 '부산광역시해운대구반송동',
 '부산광역시북구금곡동',
 '충청남도부여군부여읍',
 '경상남도사천시벌리동',
 '충청남도서산시석림동',
 '강원도속초시교동',
 '울산광역시북구화봉동',
 '강원도원주시문막읍',
 '부산광역시기장군정관읍',
 '제주특별자치도제주시아라일동',
 '제주특별자치도제주시노형동',
 '제주특별자치도제주시화북일동',
 '경상남도진주시가좌동',
 '경상남도진주시평거동',
 '경상남도창원시진해구자은동',
 '경상남도창녕군창녕읍',
 '충청남도천안시서북구백석동',
 '충청남도천안시서북구성정동',
 '충청남도천안시서북구쌍용동',
 '충청남도청양군청양읍',
 '강원도춘천시후평동',
 '강원도태백시철암동',
 '경상남도통영시도남동',
 '충청남도홍성군홍성읍',
 '제주특별자치도제주시화북일동',
 

In [86]:
juso[112][:-1]

['대구광역시', '달성군', '다사읍']

In [79]:
"아리 ".strip()

'아리'

In [57]:
bjd3

,법정동코드,법정동명,폐지여부,시군구,법정동,시군구1,시군구2,법정동1,법정동2,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,1100000000,서울특별시,존재,11000,00000,서울특별시,,,,1100000000,서울특별시,"9,565,990","4,405,638",2.1700,"4,649,442","4,916,548",0.9500
1,1111000000,서울특별시 종로구,존재,11110,00000,서울특별시,종로구,,,1111000000,서울특별시 종로구,"146,029","73,525",1.9900,"70,985","75,044",0.9500
2,1114000000,서울특별시 중구,존재,11140,00000,서울특별시,중구,,,1114000000,서울특별시 중구,"123,016","62,897",1.9600,"60,019","62,997",0.9500
3,1117000000,서울특별시 용산구,존재,11170,00000,서울특별시,용산구,,,1117000000,서울특별시 용산구,"226,378","111,650",2.0300,"108,948","117,430",0.9300
4,1120000000,서울특별시 성동구,존재,11200,00000,서울특별시,성동구,,,1120000000,서울특별시 성동구,"289,162","134,542",2.1500,"141,042","148,120",0.9500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2217,5011012200,제주특별자치도 제주시 노형동,존재,50110,12200,제주특별자치도,제주시,노형동,,5011066000,제주특별자치도 제주시 노형동,"55,385","23,087",2.4000,"27,144","28,241",0.9600
2218,5011013700,제주특별자치도 제주시 연동,존재,50110,13700,제주특별자치도,제주시,연동,,5011065000,제주특별자치도 제주시 연동,"41,910","21,203",1.9800,"20,525","21,385",0.9600
2226,5013010500,제주특별자치도 서귀포시 동홍동,존재,50130,10500,제주특별자치도,서귀포시,동홍동,,5013057000,제주특별자치도 서귀포시 동홍동,"23,205","9,512",2.4400,"11,316","11,889",0.9500
2227,5013010600,제주특별자치도 서귀포시 서홍동,존재,50130,10600,제주특별자치도,서귀포시,서홍동,,5013058000,제주특별자치도 서귀포시 서홍동,"10,925","4,483",2.4400,"5,401","5,524",0.9800


In [59]:
bjd_key = list(map(lambda x: ''.join(x.split()) , bjd3.법정동명))

In [61]:
len(set(bjd_key) & set(juso))

58

In [64]:
pop1_key = list(map(lambda x: ''.join(x.split()) , pop1.행정기관))

In [98]:
set(pop1_key) & set(juso)

{'강원도강릉시홍제동',
 '강원도속초시교동',
 '강원도속초시조양동',
 '강원도양양군강현면',
 '강원도원주시개운동',
 '강원도원주시단계동',
 '강원도원주시무실동',
 '강원도원주시문막읍',
 '강원도정선군고한읍',
 '강원도정선군정선읍',
 '강원도춘천시퇴계동',
 '강원도태백시철암동',
 '강원도평창군진부면',
 '강원도횡성군횡성읍',
 '경기도구리시갈매동',
 '경기도남양주시별내동',
 '경기도남양주시진접읍',
 '경기도남양주시퇴계원읍',
 '경기도남양주시호평동',
 '경기도성남시분당구백현동',
 '경기도성남시분당구삼평동',
 '경기도성남시수정구고등동',
 '경기도성남시중원구도촌동',
 '경기도성남시중원구성남동',
 '경기도안산시단원구신길동',
 '경기도양주시광적면',
 '경기도양주시백석읍',
 '경기도용인시기흥구보라동',
 '경기도용인시기흥구신갈동',
 '경기도의왕시고천동',
 '경기도의왕시청계동',
 '경기도평택시고덕면',
 '경기도평택시안중읍',
 '경기도화성시남양읍',
 '경상남도거제시장평동',
 '경상남도거제시하청면',
 '경상남도김해시진영읍',
 '경상남도밀양시가곡동',
 '경상남도밀양시내이동',
 '경상남도밀양시삼문동',
 '경상남도사천시사남면',
 '경상남도사천시용현면',
 '경상남도양산시물금읍',
 '경상남도양산시소주동',
 '경상남도진주시문산읍',
 '경상남도진주시충무공동',
 '경상남도진주시평거동',
 '경상남도창녕군창녕읍',
 '경상남도창원시마산회원구내서읍',
 '경상남도창원시의창구봉림동',
 '경상남도창원시진해구자은동',
 '경상남도통영시미수동',
 '경상남도함안군가야읍',
 '경상남도함안군칠원읍',
 '경상남도함양군함양읍',
 '경상북도경산시진량읍',
 '경상북도경주시건천읍',
 '경상북도고령군다산면',
 '경상북도김천시율곡동',
 '경상북도상주시함창읍',
 '경상북도예천군예천읍',
 '경상북도울진군울진읍',
 '광주광역시광산구신가동',
 '광주광역시북구신용동',
 '광주광역시북구용봉동',
 '대구광역시달서구장기동',
 '

In [97]:
set(juso) - set(pop1_key)

{'강원도강릉시교동',
 '강원도강릉시입암동',
 '강원도강릉시포남동',
 '강원도강릉시회산동',
 '강원도원주시반곡동',
 '강원도춘천시우두동',
 '강원도춘천시후평동',
 '경기도광명시소하동',
 '경기도군포시당동',
 '경기도군포시도마교동',
 '경기도성남시수정구창곡동',
 '경기도안산시상록구팔곡일동',
 '경기도안성시아양동',
 '경기도양주시고암동',
 '경기도양주시고읍동',
 '경기도양주시광사동',
 '경기도양주시만송동',
 '경기도양주시옥정동',
 '경기도용인시기흥구공세동',
 '경기도용인시기흥구영덕동',
 '경기도용인시기흥구중동',
 '경기도용인시기흥구청덕동',
 '경기도용인시수지구상현동',
 '경기도용인시수지구죽전동',
 '경기도의왕시삼동',
 '경기도의왕시포일동',
 '경기도이천시갈산동',
 '경기도평택시비전동',
 '경기도평택시이충동',
 '경상남도김해시구산동',
 '경상남도김해시부곡동',
 '경상남도김해시삼문동',
 '경상남도김해시율하동',
 '경상남도김해시장유동',
 '경상남도사천시벌리동',
 '경상남도사천시용강동',
 '경상남도양산시남부동',
 '경상남도진주시가좌동',
 '경상남도창원시의창구도계동',
 '경상남도통영시도남동',
 '경상북도경산시사동',
 '경상북도경산시평산동',
 '경상북도구미시옥계동',
 '경상북도김천시덕곡동',
 '경상북도김천시삼락동',
 '경상북도영주시가흥동',
 '경상북도영천시망정동',
 '경상북도포항시북구양덕동',
 '광주광역시광산구선암동',
 '광주광역시광산구쌍암동',
 '광주광역시광산구장덕동',
 '광주광역시광산구흑석동',
 '광주광역시남구노대동',
 '광주광역시남구백운동',
 '광주광역시남구임암동',
 '광주광역시남구행암동',
 '광주광역시북구연제동',
 '광주광역시북구용두동',
 '대구광역시달서구대곡동',
 '대전광역시대덕구법동',
 '대전광역시서구관저동',
 '대전광역시서구월평동',
 '대전광역시유성구반석동',
 '대전광역시유성구봉명동',
 '대전광역시유성구봉산동',
 '대전광역시유성구상대동',
 '대전